In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
#pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

In [3]:
train.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [4]:
test.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

* Contamos con un total de:
> * 16947 Registros en nuestro set de datos de *train*. 
> * 2551 Registros en nuestro set de datos de *test*.
> * Los 2551 registros a predecir en test, representan un 15% en cantidad de registros que tenemos en train (16947). En cantidad de oportunidades como son los porcentajes?

* Cuántas Oportunidades distintas tengo en *train* y *test*?

In [7]:
len(set(train.Opportunity_Name))

9841

In [8]:
len(set(test.Opportunity_Name))

1567

> * Se observa que las oportunidades a predecir en el set de *test* representan un 15,9% en cantidad de oportunidades que tenemos en *train*, manteniendo casi la misma proporción que teníamos con la cantidad de registros.

 * Veamos si existen registros duplicadas y para eso comparamos todos los valores existentes cada una de las filas sin tener en cuenta la columna *ID* que es el id único del registro o fila.

In [9]:
train[train.duplicated(subset=list(train.columns)[1:],keep=False)].shape[0]

619

In [10]:
test[test.duplicated(subset=list(test.columns)[1:],keep=False)].shape[0]

187

* Tenemos 619 Registros de *train* y 187 Registros de *test* que se repiten en nuestro set de datos.

* Respecto a esto: en nuestro set de *test* no podemos eliminar duplicados ya que son a todos esos registros a los que debemos predecir el *Stage*. En cambio en *train* podría pasar que filtrando estos casos repetidos podamos llegar a tener una mejor performance en nuestras predicciones ya que estaré entrenando con registros distintos y saco duplicados que no aportarían a priori información al entrenamiento sino mas bien algo de ruido. (Probar y en el Informe comentar si hay alguna mejora filtrando duplicados)

* Si nos quedamos en *train* solo con la primera aparición de esos Registros duplicados, cuantos Registros en total debería filtrar?

In [11]:
train[train.duplicated(subset=list(train.columns)[1:],keep='first')].shape[0]

448

* A cuántas oportunidades distintas corresponden esos registros duplicados?

In [12]:
len(set(train[train.duplicated(subset=list(train.columns)[1:],keep=False)].Opportunity_Name))

131

In [13]:
# Vamos a eliminar duplicados excepto los Registros que correspondad a la primera aparición.
#train = train.drop_duplicates(subset=list(train.columns)[1:],keep='first') 
#len(set(train.ID))

* Si realizamos el filtrado de Registros repetidos en *train*, conservamos las mismas cantidades de Oportunidades (9841) pero pasamos de 16947 a 16499 Registros, osea que eliminamos 448 Registros duplicados que corresponden a un 2,64% de los registros iniciales. (Probar performance con este filtro)

### Limpieza de Regiones y Territorios 
##### (*la limpieza aplicada en train debe ser la misma que en test*)

In [14]:
train.Region.value_counts()

Japan          4892
EMEA           4664
Americas       3945
APAC           3262
Middle East     184
Name: Region, dtype: int64

In [15]:
test.Region.value_counts()

Americas       1016
EMEA            838
APAC            482
Japan           201
Middle East      14
Name: Region, dtype: int64

In [16]:
# Para los casos en que tenga a Japon como Region y a None como Territory le asigno Japón a Territory, sinó lo dejo como está
train['Territory'] = train.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)
test['Territory'] = test.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)

In [17]:
# Dejo Japón solo como territorio y como Región le asigno APAC (Asia-Pacífico)
train.loc[train['Region']=='Japan','Region']='Asia-Pacífico'
test.loc[test['Region']=='Japan','Region']='Asia-Pacífico'

In [18]:
# Cambio el nombre de la región Americas por América
train.loc[train['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
train.loc[train['Region']=='EMEA','Region']='Europa'
train.loc[train['Region']=='Middle East','Region']='Oriente Medio'
train.loc[train['Region']=='APAC','Region']='Asia-Pacífico'

# Lo mismo con test
test.loc[test['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
test.loc[test['Region']=='EMEA','Region']='Europa'
test.loc[test['Region']=='Middle East','Region']='Oriente Medio'
test.loc[test['Region']=='APAC','Region']='Asia-Pacífico'

In [19]:
train.Region.value_counts()

Asia-Pacífico    8154
Europa           4664
América          3945
Oriente Medio     184
Name: Region, dtype: int64

In [20]:
test.Region.value_counts()

América          1016
Europa            838
Asia-Pacífico     683
Oriente Medio      14
Name: Region, dtype: int64

* Un dato interesante a destacar es que en train la región predominante es Asia-Pacífico y en test en América siendo Asia-Pacífico la que ocupa la tercera posición. Este dato no es menor ya que puede que según cada región puedo sectorizar los entrenemientos y predicciones. Vamos a probar esta segmentación de predicciones para ver si obtenemos mejor performance que entrenar todo junto.

In [21]:
## Arreglamos la asignación de los territorios a sus correspondientes Regiones
train.loc[train['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
train.loc[train['Territory']=='KSA','Territory']= 'Arabia Saudita'
train.loc[(train['Territory']=='Arabia Saudita')|(train['Territory']=='Jordan')|(train['Territory']=='UAE (Dubai)')\
       |(train['Territory']=='Egypt')|(train['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
train.loc[(train['Territory']=='South America')|(train['Territory']=='SE America')|(train['Territory']=='SW America')\
       |(train['Territory']=='Central America'),'Region']= 'América'
train.loc[train['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

## Lo mismo para test
test.loc[test['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
test.loc[test['Territory']=='KSA','Territory']= 'Arabia Saudita'
test.loc[(test['Territory']=='Arabia Saudita')|(test['Territory']=='Jordan')|(test['Territory']=='UAE (Dubai)')\
       |(test['Territory']=='Egypt')|(test['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
test.loc[(test['Territory']=='South America')|(test['Territory']=='SE America')|(test['Territory']=='SW America')\
       |(test['Territory']=='Central America'),'Region']= 'América'
test.loc[test['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

### Agregamos el Hemisferio según corresponda a cada Territorio.

In [22]:
# Voy a Clasificar los Territorios según los Hemisferios que pertenecen
mis_kw_train = train['Territory']
mis_kw_test = test['Territory']

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {"Hemisferio Norte": ["NW America","Japan", "India","Italy","Germany","United Kingdom","UAE (Dubai)","Greece"\
                                     ,"Switzerland","Austria","NE America","Belgium","Tunisia",'SE America',"France","Spain","Singapore"\
                                     ,"Philippines","SW America",'Denmark', 'Central America', 'Poland', 'Norway', 'Thailand'\
                                     ,'Portugal', 'South Korea','Netherlands','Kuwait','Arabia Saudita','Cambodia', 'China (PRC)', 'KSA', 'Ireland'\
                                     , 'Jordan','Cyprus', 'Morocco', 'Turkey', 'Sweden', 'Monaco', 'Kazakhstan','Romania', 'Hungary'\
                                     , 'Taiwan','Macedonia', 'Ukraine', 'Czech Republic', 'Belarus','Vietnam', 'Saudi Arabia'\
                                     ,'Bosnia & Herzegovina', 'Kosovo', 'Luxembourg', 'Bulgaria','UAE (Others)', 'Armenia', 'Croatia'\
                                     , 'Malaysia', 'Georgia','Estonia', 'Pakistan', 'Canada', 'Central US', 'Carribean','Malta'\
                                     , 'Albania', 'Oman', 'Burma', 'Slovenia','Egypt','None','Andorra','Finland','Mongolia']\
               ,"Hemisferio Sur": ["Australia",'South America','Indonesia','South East Asia','Solomon Islands','New Zealand']\
             }
key_depuradas_train=[]
key_depuradas_test=[]
keyword_processor.add_keywords_from_dict(keyword_dict)

for w in mis_kw_train:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_train.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_train.append(w)
            
for w in mis_kw_test:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_test.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_test.append(w)     

In [23]:
len(key_depuradas_train)

16947

In [24]:
len(key_depuradas_test)

2551

In [25]:
len(train['Territory'])

16947

In [26]:
len(test['Territory'])

2551

In [27]:
train['Hemisferio'] = key_depuradas_train
test['Hemisferio'] = key_depuradas_test

In [28]:
### Analizar hemisferios
#¿Filtro los hemisferios para los cuales no tenía un territorio definido?
#df = df.loc[(df['Hemisferio']!='None'),:]

In [29]:
train[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
16942,8781,Europa,Austria,Hemisferio Norte
16943,8786,Europa,Austria,Hemisferio Norte
16944,8792,Europa,Austria,Hemisferio Norte
16945,28561,América,NE America,Hemisferio Norte
16946,28318,América,NE America,Hemisferio Norte


In [30]:
test[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
2546,16345,Oriente Medio,Arabia Saudita,Hemisferio Norte
2547,15218,América,SE America,Hemisferio Norte
2548,15224,América,SE America,Hemisferio Norte
2549,7286,América,NE America,Hemisferio Norte
2550,11263,Europa,Belgium,Hemisferio Norte


In [31]:
set(test.Territory)

{'Andorra',
 'Arabia Saudita',
 'Australia',
 'Austria',
 'Belgium',
 'Bulgaria',
 'Cambodia',
 'Canada',
 'Carribean',
 'Central US',
 'China (PRC)',
 'Cyprus',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'India',
 'Indonesia',
 'Italy',
 'Japan',
 'Jordan',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mongolia',
 'NE America',
 'NW America',
 'Netherlands',
 'New Zealand',
 'None',
 'Norway',
 'Philippines',
 'Poland',
 'Portugal',
 'SE America',
 'SW America',
 'Singapore',
 'South America',
 'Spain',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'Thailand',
 'United Kingdom'}

In [32]:
set(train.Territory)

{'Albania',
 'Arabia Saudita',
 'Armenia',
 'Australia',
 'Austria',
 'Belarus',
 'Belgium',
 'Bosnia & Herzegovina',
 'Bulgaria',
 'Burma',
 'Cambodia',
 'Canada',
 'Carribean',
 'Central America',
 'Central US',
 'China (PRC)',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Egypt',
 'Estonia',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Hungary',
 'India',
 'Indonesia',
 'Ireland',
 'Italy',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kosovo',
 'Kuwait',
 'Lebanon',
 'Luxembourg',
 'Macedonia',
 'Malaysia',
 'Malta',
 'Monaco',
 'Morocco',
 'NE America',
 'NW America',
 'Netherlands',
 'New Zealand',
 'None',
 'Norway',
 'Oman',
 'Pakistan',
 'Philippines',
 'Poland',
 'Portugal',
 'Romania',
 'SE America',
 'SW America',
 'Singapore',
 'Slovenia',
 'Solomon Islands',
 'South America',
 'South East Asia',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'Thailand',
 'Tunisia',
 'Turkey',
 'UAE (Dubai)',
 'UAE (Others)',
 'Ukraine',
 'United Kingdom',
 'Vietnam'

### Agregamos La estación del año en que se crean las oportunidades en funcion del Hemisferio del Terrirorio al que pertenecen.

> * (podría agregar estaciones de entrega o de ultima modificación para tener más features)

In [33]:
train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

In [34]:
def encuentra_Estacion_N(date): 
    year = str(date.year)
    seasons = {'Primavera': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Verano': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Otono': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Verano']:
        return 'Verano'
    if date in seasons['Otono']:
        return 'Otono'
    else:
        return 'Invierno'   
    
def encuentra_Estacion_S(date): 
    year = str(date.year)
    seasons = {'Otono': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Invierno': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Primavera': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Invierno']:
        return 'Invierno'
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Otono']:
        return 'Otono'
    else:
        return 'Verano'     

In [35]:
train.loc[train['Hemisferio']=='Hemisferio Norte','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
train.loc[train['Hemisferio']=='Hemisferio Sur','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

test.loc[test['Hemisferio']=='Hemisferio Norte','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
test.loc[test['Hemisferio']=='Hemisferio Sur','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

In [36]:
train[['Opportunity_ID','Opportunity_Created_Date','Hemisferio','Estación']].sample(50)

,Opportunity_ID,Opportunity_Created_Date,Hemisferio,Estación
16814,12736,2015-11-17,Hemisferio Norte,Otono
10194,6389,2017-10-24,Hemisferio Sur,Primavera
10906,6724,2017-11-25,Hemisferio Norte,Otono
5110,3054,2016-11-15,Hemisferio Norte,Otono
14708,9150,2018-08-10,Hemisferio Norte,Verano
8924,5559,2017-07-11,Hemisferio Norte,Verano
7145,4398,2017-05-15,Hemisferio Norte,Primavera
5158,3080,2016-11-17,Hemisferio Norte,Otono
1138,623,2017-02-02,Hemisferio Norte,Invierno
13272,8079,2018-05-01,Hemisferio Norte,Primavera


In [37]:
train.Estación.value_counts(normalize=True)

Otono        0.270522
Primavera    0.265447
Verano       0.233992
Invierno     0.230038
Name: Estación, dtype: float64

In [38]:
test.Estación.value_counts(normalize=True)

Invierno     0.684829
Primavera    0.226970
Otono        0.051352
Verano       0.036848
Name: Estación, dtype: float64

In [39]:
train.shape

(16947, 54)

### Precio de las oportunidades

* Vamos a crear un feature con el valor total de ventas de las oportunidades (**Total_Taxable_Amount**) en una misma moneda (**dólares estadounidenses**) para que la comparación entre valores sean coherentes. También cabe destacar que en *test* tenemos casos de **Total_Taxable_Amount = 0** pero no tenemos casos de **Total_Amount = 0**, con lo cual calcularemos el precio final de cada **oportunidad** sumando dichos valores de Total_Amount para cada una de ellas. Esto lo vamos a realizar también en *train* pero tener en cuenta que train además cuenta con registros que contienen simultaneamente **Total_Taxable_Amount y Total_Taxable_Amount igual a 0**, (específicamente 39 casos que no se dan en test) con lo cual debo tomar la mejor decición, ya sea filtrando esos casos de train o imputando algún valor a Total_Taxable_Amount.    

In [40]:
train.loc[(train['Total_Amount']==0) & ((train['Total_Taxable_Amount']==0)),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación
464,28260,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/30/2015,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
579,25679,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono
580,25718,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono
586,15710,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,12/9/2016,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono
1130,17845,Europa,Germany,1,0,1,0,0,Bureaucratic_Code_5,10/29/2013,...,NaT,0,EUR,0.0,EUR,0.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Invierno
1437,22806,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
1705,14772,Oriente Medio,None,1,0,1,0,0,Bureaucratic_Code_5,11/15/2016,...,NaT,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
1904,8897,Europa,United Kingdom,1,0,1,0,0,Bureaucratic_Code_5,1/20/2016,...,NaT,5,EUR,0.0,EUR,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
2083,10670,América,SE America,1,1,1,1,0,Bureaucratic_Code_4,2/13/2016,...,NaT,0,USD,0.0,USD,0.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Invierno
2823,14860,Europa,None,1,0,1,0,0,Bureaucratic_Code_5,5/9/2016,...,NaT,5,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Primavera


In [41]:
train = train.loc[(train['Total_Amount']!=0) | ((train['Total_Taxable_Amount']!=0)),:]

In [42]:
train.shape

(16908, 54)

In [43]:
test.loc[test['Total_Amount']==0,:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación


In [44]:
test.loc[(test['Total_Amount']==0) & ((test['Total_Taxable_Amount']==0)),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación


In [45]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,NaT,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,NaT,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno


In [46]:
train_monto_total=train.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
train_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
train_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,0,5272800.0
1,1,48230.0
2,2,83865.6
3,3,7421881.5
4,4,13357192.5


In [47]:
test_monto_total=test.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
test_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
test_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,10689,367419.0
1,10690,757783.5
2,10691,21037.5
3,10692,2169106.5
4,10693,5752.5


In [48]:
train=pd.merge(train,train_monto_total,how='inner',on='Opportunity_ID')

In [49]:
test=pd.merge(test,test_monto_total,how='inner',on='Opportunity_ID')

In [50]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,1332347.5
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,1332347.5


In [51]:
train.ASP_Currency.value_counts()

USD    7338
JPY    4520
EUR    4489
AUD     556
GBP       5
Name: ASP_Currency, dtype: int64

In [52]:
train.loc[train['ASP_Currency']=='GBP',:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
2290,20861,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1837080.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2291,20878,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1801800.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2292,20881,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1798290.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2293,20884,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1837080.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
3565,17651,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,10/15/2013,...,2,GBP,776081.25,EUR,9.043541e+05,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Verano,776081.25


In [53]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,5272800.0
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otono,48230.0
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otono,83865.6
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,7421881.5
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,13357192.5


In [54]:
def conver_EUR_dolares(monto): 
        return monto*1.131   
    
def conver_JPY_dolares(monto): 
        return monto*0.00899 

def conver_AUD_dolares(monto): 
        return monto*0.7076 
    
def conver_GBP_dolares(monto): 
        return monto*1.3181 

def conver_USD_dolares(monto): 
        return monto*1

In [55]:
train.loc[train['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

In [56]:
test.loc[test['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

### Total Amount en dólares

In [57]:
train.loc[train['Total_Amount_Currency']=='EUR','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='EUR','Total_Amount']\
.map(conver_EUR_dolares)
train.loc[train['Total_Amount_Currency']=='JPY','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='JPY','Total_Amount']\
.map(conver_JPY_dolares)
train.loc[train['Total_Amount_Currency']=='AUD','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='AUD','Total_Amount']\
.map(conver_AUD_dolares)
train.loc[train['Total_Amount_Currency']=='GBP','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='GBP','Total_Amount']\
.map(conver_GBP_dolares)
train.loc[train['Total_Amount_Currency']=='USD','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='USD','Total_Amount']\
.map(conver_USD_dolares)

test.loc[test['Total_Amount_Currency']=='EUR','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='EUR','Total_Amount']\
.map(conver_EUR_dolares)
test.loc[test['Total_Amount_Currency']=='JPY','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='JPY','Total_Amount']\
.map(conver_JPY_dolares)
test.loc[test['Total_Amount_Currency']=='AUD','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='AUD','Total_Amount']\
.map(conver_AUD_dolares)
test.loc[test['Total_Amount_Currency']=='GBP','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='GBP','Total_Amount']\
.map(conver_GBP_dolares)
test.loc[test['Total_Amount_Currency']=='USD','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='USD','Total_Amount']\
.map(conver_USD_dolares)

In [58]:
train.head(20)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,5272800.0,5.963537e+06,5.963537e+06
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otono,48230.0,5.454813e+04,5.454813e+04
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otono,83865.6,8.386560e+04,8.386560e+04
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,7421881.5,7.421882e+06,7.421882e+06
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,13357192.5,1.335719e+07,1.335719e+07
5,16802,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,14838277.5,USD,14838277.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,14838277.5,1.483828e+07,1.483828e+07
6,16799,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,2659494.6,USD,2659494.6,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,2659494.6,2.659495e+06,2.659495e+06
7,27455,América,NW America,1,1,1,1,0,Bureaucratic_Code_4,4/21/2015,...,50688.0,USD,50688.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otono,50688.0,5.068800e+04,5.068800e+04
8,24353,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,15600.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,470400.0,4.228896e+03,1.402440e+02
9,24355,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,4400.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,470400.0,4.228896e+03,3.955600e+01


In [59]:
test.head(10)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares
0,6140,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,162240.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.889,183493.44
1,6146,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,78624.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.889,88923.744
2,6151,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,126555.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.889,143133.705
3,6118,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,243360.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,275240.16
4,6124,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,157248.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,177847.488
5,6129,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,126555.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,143133.705
6,6135,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,118156.5,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,133635.0015
7,6218,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,EUR,112464.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,127196.784
8,11530,América,NE America,1,1,0,0,0,Bureaucratic_Code_4,2/16/2019,...,USD,21037.5,USD,21037.5,Prod_Category_A_None,Hemisferio Norte,Primavera,21037.5,21037.5,21037.5
9,20314,América,NW America,1,1,1,0,0,Bureaucratic_Code_2,7/27/2013,...,USD,342490.5,USD,2169106.5,Prod_Category_A_None,Hemisferio Norte,Primavera,2169106.5,2169106.5,342490.5


In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16908 entries, 0 to 16907
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16908 non-null  int64         
 1   Region                              16908 non-null  object        
 2   Territory                           16908 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16908 non-null  int64         
 4   Pricing, Delivery_Terms_Approved    16908 non-null  int64         
 5   Bureaucratic_Code_0_Approval        16908 non-null  int64         
 6   Bureaucratic_Code_0_Approved        16908 non-null  int64         
 7   Submitted_for_Approval              16908 non-null  int64         
 8   Bureaucratic_Code                   16908 non-null  object        
 9   Account_Created_Date                16908 non-null  object        
 10  Source                

In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551 entries, 0 to 2550
Data columns (total 56 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   object        
 2   Territory                           2551 non-null   object        
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64         
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64         
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64         
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64         
 7   Submitted_for_Approval              2551 non-null   int64         
 8   Bureaucratic_Code                   2551 non-null   object        
 9   Account_Created_Date                2551 non-null   object        
 10  Source                  

In [62]:
train.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()

Opportunity_ID monto_en_dolares                              \
                                 max           min          mean   
0                 0       5963536.80  5.963537e+06  5.963537e+06   
1                 1         54548.13  5.454813e+04  5.454813e+04   
2                 2         83865.60  8.386560e+04  8.386560e+04   
3                 3       7421881.50  7.421882e+06  7.421882e+06   
4                 4      13357192.50  1.335719e+07  1.335719e+07   
...             ...              ...           ...           ...   
9800          12799        454322.70  4.543227e+05  4.543227e+05   
9801          12800      10751580.00  1.058092e+07  1.066625e+07   
9802          12801        116888.85  1.052000e+05  1.129926e+05   
9803          12802       2346796.88  2.346797e+06  2.346797e+06   
9804          12803      25603200.00  2.560320e+07  2.560320e+07   

                     
                std  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
...             ...  
9800            NaN  
9801  120674.843277  
9802    6748.580901  
9803            NaN  
9804            NaN  

[9805 rows x 5 columns]

In [63]:
test.total_en_dolares=test.total_en_dolares.astype('float')
test.monto_en_dolares=test.monto_en_dolares.astype('float')

In [64]:
train_monto=train.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()
test_monto=test.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()

In [65]:
train_monto

Opportunity_ID monto_en_dolares                              \
                                 max           min          mean   
0                 0       5963536.80  5.963537e+06  5.963537e+06   
1                 1         54548.13  5.454813e+04  5.454813e+04   
2                 2         83865.60  8.386560e+04  8.386560e+04   
3                 3       7421881.50  7.421882e+06  7.421882e+06   
4                 4      13357192.50  1.335719e+07  1.335719e+07   
...             ...              ...           ...           ...   
9800          12799        454322.70  4.543227e+05  4.543227e+05   
9801          12800      10751580.00  1.058092e+07  1.066625e+07   
9802          12801        116888.85  1.052000e+05  1.129926e+05   
9803          12802       2346796.88  2.346797e+06  2.346797e+06   
9804          12803      25603200.00  2.560320e+07  2.560320e+07   

                     
                std  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
...             ...  
9800            NaN  
9801  120674.843277  
9802    6748.580901  
9803            NaN  
9804            NaN  

[9805 rows x 5 columns]

In [66]:
test_monto

Opportunity_ID monto_en_dolares                                        
                                 max          min         mean           std
0             10689      183493.4400   88923.7440  138516.9630  47453.583431
1             10690      275240.1600  127196.7840  171410.6277  61244.287928
2             10691       21037.5000   21037.5000   21037.5000           NaN
3             10692      456654.0000  228327.0000  361517.7500  85939.160738
4             10693        5752.5000    5752.5000    5752.5000           NaN
...             ...              ...          ...          ...           ...
1562          12364      124740.0000   23010.0000   73875.0000  71933.972850
1563          12365       45054.9000   45054.9000   45054.9000           NaN
1564          12366      100122.0000  100122.0000  100122.0000           NaN
1565          12367      143220.0000  143220.0000  143220.0000           NaN
1566          12368      985124.5248  985124.5248  985124.5248           NaN

[1567 rows x 5 columns]

In [67]:
train_monto.columns

MultiIndex([(  'Opportunity_ID',     ''),
            ('monto_en_dolares',  'max'),
            ('monto_en_dolares',  'min'),
            ('monto_en_dolares', 'mean'),
            ('monto_en_dolares',  'std')],
           )

In [68]:
train_monto.columns = ['Opportunity_ID','monto_en_dolares_max','monto_en_dolares_min','monto_en_dolares_mean','monto_en_dolares_std']
test_monto.columns = ['Opportunity_ID','monto_en_dolares_max','monto_en_dolares_min','monto_en_dolares_mean','monto_en_dolares_std']

In [69]:
train=pd.merge(train,train_monto,how='inner',on='Opportunity_ID')
test=pd.merge(test,test_monto,how='inner',on='Opportunity_ID')

In [70]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Prod_Category_A_None,Hemisferio Norte,Otono,5272800.0,5963536.80,5963536.80,5963536.80,5963536.80,5963536.80,NaN
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Prod_Category_A_None,Hemisferio Norte,Otono,48230.0,54548.13,54548.13,54548.13,54548.13,54548.13,NaN
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Prod_Category_A_None,Hemisferio Norte,Otono,83865.6,83865.60,83865.60,83865.60,83865.60,83865.60,NaN
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Prod_Category_A_None,Hemisferio Norte,Otono,7421881.5,7421881.50,7421881.50,7421881.50,7421881.50,7421881.50,NaN
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Prod_Category_A_None,Hemisferio Norte,Otono,13357192.5,13357192.50,13357192.50,13357192.50,13357192.50,13357192.50,NaN


In [71]:
test.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std
0,6140,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,183493.440,183493.44,88923.744,138516.9630,47453.583431
1,6146,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,88923.744,183493.44,88923.744,138516.9630,47453.583431
2,6151,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415550.8890,143133.705,183493.44,88923.744,138516.9630,47453.583431
3,6118,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,275240.160,275240.16,127196.784,171410.6277,61244.287928
4,6124,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053.1385,177847.488,275240.16,127196.784,171410.6277,61244.287928


### Stage

In [73]:
train.Stage.value_counts()

Closed Won       9529
Closed Lost      7315
Proposal           35
Negotiation        18
Qualification      11
Name: Stage, dtype: int64

* En el set de datos de *train* se observa que en la variable Stage, además de los estados que indican si una oportunidad resultó exitosa o no (**estados finales**), tenemos otros 3 estados (Proposal, Negotiation y Qualification) que corresponden a **estados intermedios** que tienen las oportunidades dentro del pipeline que desarrolla la empresa.

* Como con esta variable se estará luego tratando de predecir la “Probabilidad de éxito” para cada oportunidad en el set de *test*, o sea ¿Cuál es la probabilidad de que la oportunidad se convierta en un caso Closed Won? . Que decisión debemos tomar con dichos estados? Dichos estados al no tener una resolución, debemos asignarle arbitrariamente algún estado final? Debemos filtrar estos casos?

> * Finalmente luego de analizar esos registros con estos estados intermedios y verificar que corresponen a oportunidades que no tuvieron aún un desenlase (exitoso o no exitoso) y tenindo en cuenta que corresponden a unos pocos casos, tomamos la decisión de filtrar estos registros y quedarnos solamente con los que tienen un estado final definido.

In [74]:
train=train.loc[(train['Stage']!='Proposal') & (train['Stage']!='Negotiation') & (train['Stage']!='Qualification'),:]

In [75]:
train.Stage.value_counts()

Closed Won     9529
Closed Lost    7315
Name: Stage, dtype: int64

### Cantidad de Items de cada Oportunidad

In [76]:
Op_cant_train = train.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_test = test.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_train.rename(columns = {'ID': 'Cantidad de Items Asociados'}, inplace = True)
Op_cant_test.rename(columns = {'ID': 'Cantidad de Items Asociados'}, inplace = True)

train=pd.merge(Op_cant_train,train,on='Opportunity_ID')
test=pd.merge(Op_cant_test,test,on='Opportunity_ID')

In [77]:
train[['ID','Region','Opportunity_ID','Cantidad de Items Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad de Items Asociados
7939,14100,América,4956,1
10737,21917,Asia-Pacífico,6654,16
5652,11389,América,3496,1
4947,22649,Asia-Pacífico,2937,9
3692,24513,Asia-Pacífico,2063,6


In [78]:
test[['ID','Region','Opportunity_ID','Cantidad de Items Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad de Items Asociados
1802,13292,Europa,11913,3
361,13774,Europa,10900,2
1578,11885,Europa,11788,1
1844,19770,Europa,11944,1
560,9122,Asia-Pacífico,11019,2


### Precio total en dolares por Items de la Oportunidad

In [79]:
train['precio_por_items'] = train['total_en_dolares']/train['Cantidad de Items Asociados']

In [80]:
test['precio_por_items'] = test['total_en_dolares']/test['Cantidad de Items Asociados']

In [81]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Hemisferio,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,precio_por_items
0,10689,3,6140,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,367419.0,415550.8890,183493.440,183493.44,88923.744,138516.9630,47453.583431,138516.9630
1,10689,3,6146,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,367419.0,415550.8890,88923.744,183493.44,88923.744,138516.9630,47453.583431,138516.9630
2,10689,3,6151,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,367419.0,415550.8890,143133.705,183493.44,88923.744,138516.9630,47453.583431,138516.9630
3,10690,5,6118,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,757783.5,857053.1385,275240.160,275240.16,127196.784,171410.6277,61244.287928,171410.6277
4,10690,5,6124,Europa,Germany,1,1,1,1,0,...,Hemisferio Norte,Primavera,757783.5,857053.1385,177847.488,275240.16,127196.784,171410.6277,61244.287928,171410.6277


### Media del Total Amount en dolares por items de Oportunidad

In [82]:
train['media_por_items'] = train['monto_en_dolares_mean']/train['Cantidad de Items Asociados']
test['media_por_items'] = test['monto_en_dolares_mean']/test['Cantidad de Items Asociados']

### Calculamos la frecuencia de aparición para los valores coreespondientes a columnas categóricas para las distintas oportunidades

In [83]:
df = pd.concat([train,test])

In [84]:
#Limpiamos los nombres de las columnas para que contengan solo valores alfanuméricos
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [85]:
mis_brand=pd.crosstab(df.Opportunity_ID,df.Brand).reset_index()
df=pd.merge(df, mis_brand,on='Opportunity_ID' ,how='inner')

In [86]:
mis_Burocratic_code=pd.crosstab(df.Opportunity_ID,df.Bureaucratic_Code).reset_index()
df=pd.merge(df, mis_Burocratic_code,on='Opportunity_ID' ,how='inner')

In [87]:
mis_Product_Name=pd.crosstab(df.Opportunity_ID,df.Product_Name).reset_index()
df=pd.merge(df, mis_Product_Name,on='Opportunity_ID' ,how='inner')

In [88]:
mis_Opportunity_Owner=pd.crosstab(df.Opportunity_ID,df.Opportunity_Owner).reset_index()
df=pd.merge(df, mis_Opportunity_Owner,on='Opportunity_ID' ,how='inner')

In [89]:
mis_Account_Owner=pd.crosstab(df.Opportunity_ID,df.Account_Owner).reset_index()
df=pd.merge(df, mis_Account_Owner,on='Opportunity_ID' ,how='inner')

In [90]:
mis_Product_Family=pd.crosstab(df.Opportunity_ID,df.Product_Family).reset_index()
df=pd.merge(df, mis_Product_Family,on='Opportunity_ID' ,how='inner')

In [91]:
mis_Account_Name=pd.crosstab(df.Opportunity_ID,df.Account_Name).reset_index()
df=pd.merge(df, mis_Account_Name,on='Opportunity_ID' ,how='inner')

### Fechas

In [92]:
df['Planned_Delivery_Start_Date']=pd.to_datetime(df['Planned_Delivery_Start_Date'])
df['Planned_Delivery_End_Date']=pd.to_datetime(df['Planned_Delivery_End_Date'])
df['year_entrega']=df['Planned_Delivery_End_Date'].dt.year
df['mes_entrega']=df['Planned_Delivery_End_Date'].dt.month
df['Opportunity_Created_Date']=pd.to_datetime(df['Opportunity_Created_Date'])
df['Last_Activity']=pd.to_datetime(df['Last_Activity'])
df['Account_Created_Date']=pd.to_datetime(df['Account_Created_Date'])
df['Quote_Expiry_Date']=pd.to_datetime(df['Quote_Expiry_Date'])
df['Last_Modified_Date']=pd.to_datetime(df['Last_Modified_Date'])
#df.loc[df['Quote_Expiry_Date'].isnull(),'Quote_Expiry_Date']=pd.datetime.now()
#df.loc[df['Quote_Expiry_Date'].isnull(),'Quote_Expiry_Date']=pd.datetime(2014, 1, 1)
#df['Quote_Expiry_Date']=pd.to_datetime(df['Quote_Expiry_Date'])

In [93]:
df['tiempo_plan_end_star']=df['Planned_Delivery_End_Date']-df['Planned_Delivery_Start_Date']
df['tiempo_plan_end_star'] = df['tiempo_plan_end_star'].dt.total_seconds()/36000

#df['tiempo_cuenta_oportunidad']=df['Account_Created_Date']-df['Opportunity_Created_Date']
#df['tiempo_cuenta_oportunidad']=df['tiempo_cuenta_oportunidad'].dt.total_seconds()/36000

#df['tiempo_presupuesto_entrega']=df['Planned_Delivery_Start_Date']-df['Quote_Expiry_Date']
#df['tiempo_presupuesto_entrega']=df['tiempo_presupuesto_entrega'].dt.total_seconds()/360000

#df['tiempo_modificacion_creacion']=df['Opportunity_Created_Date']-df['Last_Modified_Date']
#df['tiempo_modificacion_creacion']=df['tiempo_modificacion_creacion'].dt.total_seconds()/36000
#train['year_entrega']=train['Planned_Delivery_End_Date'].dt.year
#train['mes_entrega']=train['Planned_Delivery_End_Date'].dt.month
#test['year_entrega']=test['Planned_Delivery_End_Date'].dt.year
#test['mes_entrega']=test['Planned_Delivery_End_Date'].dt.month

In [94]:
train =train[['ID']]
train = pd.merge(train,df, on= 'ID', how = 'inner') 
test =test[['ID']]
test = pd.merge(test,df, on= 'ID', how = 'inner') 

In [95]:
train.to_csv("train_listo.csv", index=False)
test.to_csv("test_listo.csv", index=False)